## RAG 구현하기 => 검색 / 증강/ 생성   
(Retrieval Augment Generation)
=> 새로운 정보를 토대로 언어를 생성
=> LLM이 언어를 생성하는 것을 훈련할 때 없었던 정보도, 검색(문헌정보, 웹)을 통해 증강함으로써 더 정확한 답변을 얻는 기술

[자연어처리]
- 문서 가져오기
- 문서 전처리
- 임베딩
- 벡터화(벡터 유사도) -> 단어의 의미를 파악할 수 있도록 하는 단계
- 검색

In [ ]:
#비정형데이터(unstructured data)를 다루는 오픈소스
#PDF, HTML, Word문서 텍스트, 이미지 데이터를 수집하고 전처리하는 용도
!pip install unstructured==0.11.6

#문장, 텍스트 및 이미지의 "임베딩"을 위한 도구
!pip install sentence-transformers==2.2.2

!pip install langchain==0.1.0

!pip install openai==1.4.0

#벡터화DB
!pip install chromadb==0.4.21

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━

##1. 직접 만든 텍스트를 마운트합니다.

In [ ]:
path = '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'

In [ ]:
from langchain.document_loaders import TextLoader

documents = TextLoader(path).load()

In [ ]:
type(documents)

list

In [ ]:
#Document라는 형태의 객체로 리스트 안에 불러와져 있습니다.
#documents는
#page_content, metadata라는 두 가지 속성을 가짐
#page_content=실제 텍스트 내용
#metadata=이 텍스트에 대한 정보
documents

[Document(page_content='임시현-김우진, 양궁 혼성 단체전 금메달 획득\n(파리=연합뉴스) 임화영 기자 = 2일(현지시간) 프랑스 파리 레쟁발리드에서 열린 2024 파리올림픽 양궁 혼성 단체 독일과의 결승전에서 승리한 한국 임시현과 김우진이 기뻐하고 있다. 2024.8.3 hwayoung7@yna.co.kr\n\n(파리=연합뉴스) 김동찬 기자 = 김우진(청주시청)과 임시현(한국체대)이 2024 파리 올림픽 양궁 2관왕에 올랐다.\n\n김우진과 임시현은 2일(한국시간) 프랑스 파리의 앵발리드에서 열린 양궁 혼성 단체전 결승에서 독일을 6-0(38-35 36-35 36-35)으로 완파했다.\n\n16강에서 대만, 8강에서 이탈리아를 차례로 꺾은 김우진과 임시현은 4강에서는 인도를 물리치고 결승까지 순항했다.\n\n\n2021년 도쿄 올림픽에서 올림픽 정식 종목이 된 양궁 혼성전에서는 우리나라가 2021년 도쿄 김제덕·안산에 이어 이번 대회 김우진·임시현이 금메달을 독식했다.\n\nADVERTISEMENT\n\n \nADVERTISEMENT\n\n이미 남녀 단체전에서 각각 금메달을 획득한 김우진과 임시현은 펜싱 오상욱(대전시청)에 이어 이번 대회 한국 선수 2관왕 대열에 합류했다.\n\n또 김우진은 최근 올림픽 3회 연속 단체전 금메달에 이어 혼성전 금메달을 더해 개인 통한 4호 금메달을 획득, 한국 선수 최다 올림픽 금메달 공동 1위가 됐다.\n\n김수녕(양궁), 진종오(사격), 전이경(쇼트트랙)이 동·하계 올림픽에서 금메달 4개를 따낸 한국 선수들이다.\n\n김우진과 임시현은 남은 개인전에서도 우승할 경우 대회 3관왕이 된다. 임시현은 지난해 항저우 아시안게임에 이어 종합 대회 2회 연속 3관왕에 도전한다.\n김원호-정나은, 아쉬운 은메달\n김원호-정나은, 아쉬운 은메달\n(파리=연합뉴스) 신준희 기자 = 2일(현지시간) 프랑스 파리 라 샤펠 아레나에서 열린 2024 파리올림픽 배드민턴 혼합복식 결승전에서 김원호-정나은 조가 중국 정쓰웨이-황야

## 2. 문서의 전처리 ==> 청크 만들기  

- "토큰화"
-> 자연어의 문맥 이해가 쉬울 수 있도록, 일정한 사이즈의 덩어리로 만들어주는 작업(청크)

In [ ]:
#재귀적인 분할기
#단락 -> 문장 -> 단어 순서로 반복하면서 분할하는 함수
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 나는 오늘 아침에 상명대학교에 와서 수업을 들었다.
# 수업을 들었다. 그리고 나는 오늘 저녁에는 신나게
def split_docs(documents, chunk_size=150, chunk_overlap=0):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                                 chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)

  return docs

In [ ]:
docs = split_docs(documents)
print(docs)

[Document(page_content='임시현-김우진, 양궁 혼성 단체전 금메달 획득', metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}), Document(page_content='(파리=연합뉴스) 임화영 기자 = 2일(현지시간) 프랑스 파리 레쟁발리드에서 열린 2024 파리올림픽 양궁 혼성 단체 독일과의 결승전에서 승리한 한국 임시현과 김우진이 기뻐하고 있다. 2024.8.3 hwayoung7@yna.co.kr', metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}), Document(page_content='(파리=연합뉴스) 김동찬 기자 = 김우진(청주시청)과 임시현(한국체대)이 2024 파리 올림픽 양궁 2관왕에 올랐다.', metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}), Document(page_content='김우진과 임시현은 2일(한국시간) 프랑스 파리의 앵발리드에서 열린 양궁 혼성 단체전 결승에서 독일을 6-0(38-35 36-35 36-35)으로 완파했다.', metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}), Document(page_content='16강에서 대만, 8강에서 이탈리아를 차례로 꺾은 김우진과 임시현은 4강에서는 인도를 물리치고 결승까지 순항했다.', metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}),

In [ ]:
for d in docs:
  print(d)

page_content='임시현-김우진, 양궁 혼성 단체전 금메달 획득' metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}
page_content='(파리=연합뉴스) 임화영 기자 = 2일(현지시간) 프랑스 파리 레쟁발리드에서 열린 2024 파리올림픽 양궁 혼성 단체 독일과의 결승전에서 승리한 한국 임시현과 김우진이 기뻐하고 있다. 2024.8.3 hwayoung7@yna.co.kr' metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}
page_content='(파리=연합뉴스) 김동찬 기자 = 김우진(청주시청)과 임시현(한국체대)이 2024 파리 올림픽 양궁 2관왕에 올랐다.' metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}
page_content='김우진과 임시현은 2일(한국시간) 프랑스 파리의 앵발리드에서 열린 양궁 혼성 단체전 결승에서 독일을 6-0(38-35 36-35 36-35)으로 완파했다.' metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}
page_content='16강에서 대만, 8강에서 이탈리아를 차례로 꺾은 김우진과 임시현은 4강에서는 인도를 물리치고 결승까지 순항했다.' metadata={'source': '/content/drive/MyDrive/천안시청 교육과정/소스코드/올림픽현황.txt'}
page_content='2021년 도쿄 올림픽에서 올림픽 정식 종목이 된 양궁 혼성전에서는 우리나라가 20

In [ ]:
len(docs)

27

## 3. 문서 임베딩하기

In [ ]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
#문서 임베딩
from langchain.embeddings import SentenceTransformerEmbeddings

#특정한 모델 양식으로 단어 임베딩을 수행함.
#사전 훈련된 모델을 사용하여 임베딩하면, 의미적 검색, 텍스트 유사도 계산
#등에 활용 가능
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 's'

In [ ]:
#"여름" = [0.002, 0.23832, -1.34982, ....] => "고차원 임베딩"

In [ ]:
from langchain.vectorstores import Chroma

#벡터스토어 : 임베딩을 통해 생성된 고차원의 벡터 데이터를
#효율적으로 저장, 조회하도록 설계된 DB
#우리가 만든 "임베딩된" 벡터를 벡터 DB에 변환하여 저장해 줌
db = Chroma.from_documents(docs, embeddings)

## 4. 검색 (질의응답)

In [ ]:
from langchain.chat_models import ChatOpenAI
model = 'gpt-3.5-turbo-0301'
llm = ChatOpenAI(model_name=model)

from langchain.chains.question_answering import load_qa_chain

#chain_type='stuff'는 일반적인 상황에 대한 질의응답
#verbose = 체인이 동작하는 단계(로그)를 상세하게 보여줌
chain = load_qa_chain(llm, chain_type='stuff', verbose=True)

In [ ]:
query = "김원호-정나은 선수는 무슨 메달을 땄어?"

#내가 넣은 문서를 숫자로 임베딩
#숫자로 변환한 내용으로 "유사도 검색"이 가능
#내가 질문한 내용과 유사한 답변 "유사도 검색"으로 찾아 답변
matching_docs = db.similarity_search(query)

answer = chain.run(input_documents=matching_docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
김수녕(양궁), 진종오(사격), 전이경(쇼트트랙)이 동·하계 올림픽에서 금메달 4개를 따낸 한국 선수들이다.

한국 유도가 최중량급에서 결승에 오른 것은 유도가 1964년 올림픽 정식 종목이 된 이후 처음이다. 이전에는 1984년 로스앤젤레스, 1988년 서울 대회 조용철과 2000년 시드니 대회 김선영의 동메달이 최고 성적이었다.

한국 배드민턴이 올림픽에서 은메달을 따낸 것은 2008년 베이징 대회 이후 16년 만이다.

2008년 베이징에서는 금, 은, 동 하나씩을 획득했고, 2012년 런던과 2016년 리우데자네이루, 2021년 도쿄에서는 동메달이 최고 성적이었다.

김원호는 1996년 애틀랜타 올림픽 혼합 복식 금메달리스트 길영아 삼성생명 감독의 아들이다.
Human: 김원호-정나은 선수는 무슨 메달을 땄어?

> Finished chain.

> Finished chain.


In [ ]:
answer

'위의 문맥에서 김원호-정나은 선수가 어떤 대회에서 무슨 메달을 딴 것에 대한 정보가 제공되지 않았습니다. 따라서 김원호-정나은 선수가 무슨 대회에서 무슨 메달을 딴 지에 대해 알 수 없습니다.'